In [7]:
from bs4 import BeautifulSoup

In [8]:
import re

In [9]:
dom = BeautifulSoup(open('./course_schedule.htm'))

In [11]:
list = dom.select('.schedule-listing li')

In [ ]:
def title(li):
    spans = li.select('span')
    return spans[1].get_text().strip()

def instructor(li):
    spans = li.select('span')
    return spans[2].get_text().strip().split('\n')[0]

def time(li):
    spans = li.select('span')
    return spans[3].get_text().strip()

temp = [(li['id'], title(li), instructor(li), time(li)) for li in list]
temp[:10]

In [33]:
import pandas as pd
schedule_df = pd.DataFrame(temp, columns = ['Number-Section', 'Name', 'Instructor', 'Time'])

In [54]:
schedule_df['Number'] = schedule_df['Number-Section'].str.extract('(\w{4}.\w{2}\d{4}).\d{3}')
schedule_df['Section'] = schedule_df['Number-Section'].str.extract('\w{4}.\w{2}\d{4}.(\d{3})')

In [105]:
schedule_df.drop(columns=['num'], inplace=True)

In [ ]:
schedule_df.drop(columns=['Num'], inplace=True)

In [109]:
schedule_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Number-Section  152 non-null    object
 1   Name            152 non-null    object
 2   Instructor      152 non-null    object
 3   Time            152 non-null    object
 4   Number          152 non-null    object
 5   Section         152 non-null    object
dtypes: object(6)
memory usage: 7.2+ KB


In [111]:
schedule_df.head(5)

,Number-Section,Name,Instructor,Time,Number,Section
0,csci-ga1144-001,PAC II*,Mohamed Zahran,T 6:00-8:30PM,csci-ga1144,001
1,csci-ga1144-002,PAC II Recitation,Gurkirat Singh Bajwa,R 7:10-8:00PM,csci-ga1144,002
2,csci-ga1170-001,Fundamental Algorithms,Yevgeniy Dodis,T 7:10-9:00PM,csci-ga1170,001
3,csci-ga1170-002,Fundamental Algorithms Recitation,Alex Bienstock,R 8:10-9:00PM,csci-ga1170,002
4,csci-ga1170-003,Fundamental Algorithms Recitation,Harish Karthikeyan,R 8:10-9:00PM,csci-ga1170,003


In [112]:
schedule_df.tail(5)

,Number-Section,Name,Instructor,Time,Number,Section
147,csci-ua0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 9:30-10:45AM,csci-ua0480,057
148,csci-ua0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,MW 3:30-4:45PM,csci-ua0480,069
149,csci-ua0480-521,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 11:00-12:15PM,csci-ua0480,521
150,csci-ua0480-522,Special Topics: Algorithmic Problem Solving,Kunal Khatri,W 3:30-4:45PM,csci-ua0480,522
151,csci-ua0480-523,Special Topics: Algorithmic Problem Solving,Samasth Ananda,W 3:30-4:45PM,csci-ua0480,523


In [113]:
schedule_df.sample(5)

,Number-Section,Name,Instructor,Time,Number,Section
143,csci-ua0479-001,Data Management and Analysis,Joseph Versoza,MW 11:00-12:15PM,csci-ua0479,001
133,csci-ua0310-006,Basic Algorithms - Recitation,Benjamin Jakubowski,W 8:00-9:15AM,csci-ua0310,006
51,csci-ga3140-001,Abstract Interpretation,Patrick Cousot,M 9:30-11:20AM,csci-ga3140,001
40,csci-ga3033-076,Special Topics:\n ...,Davi Geiger,R 7:10-9:00PM,csci-ga3033,076
37,csci-ga3033-059,Special Topics:\n ...,Anasse Bari,W 5:10-7:00PM,csci-ga3033,059


In [57]:
dom = BeautifulSoup(open('./course_catalog.htm'))

In [71]:
def points(li):
    content = li.select('p')[1].get_text()
    m = re.search('([0-9 -]+) Points', content)
    if m:
        return m[1].strip()
    
def pre_reqs(li):
    return li.select('p')[2].get_text()
    
courses = dom.select('ul.courses-listing > li')
temp_courses = [(course['id'], pre_reqs(course), points(course)) for course in courses]

In [72]:
catalog_df = pd.DataFrame(temp_courses, columns = ['Number', 'Prereqs', 'Points'])

In [73]:
catalog_df

,Number,Prereqs,Points
0,csci-ga1133,Prerequisites: None,4
1,csci-ga1144,Prerequisites: CSCI-GA 1133 or departmental pe...,4
2,csci-ga1170,Prerequisites: At least one year of experience...,3
3,csci-ga1180,Prerequisites: None,3
4,csci-ga2110,Prerequisites: Students taking this class shou...,3
...,...,...,...
95,csci-ua0897,Prerequisites: Restricted to declared computer...,1 - 4
96,csci-ua0898,Prerequisites: Restricted to declared computer...,1 - 4
97,csci-ua0997,Prerequisites: Permission of the department. D...,1 - 4
98,csci-ua0998,Prerequisites: Permission of the department. D...,1 - 4


In [97]:
catalog_df['PreReqs'] = catalog_df['Prereqs'].str.extract('\w{13}: (.*)')

In [118]:
catalog_df.drop(columns=['Prereqs'], inplace=True)

In [132]:
catalog_df = catalog_df[['Number', 'PreReqs', 'Points']]

In [133]:
catalog_df

,Number,PreReqs,Points
0,csci-ga1133,None,4
1,csci-ga1144,CSCI-GA 1133 or departmental permission.,4
2,csci-ga1170,At least one year of experience with a high-le...,3
3,csci-ga1180,None,3
4,csci-ga2110,Students taking this class should already have...,3
...,...,...,...
95,csci-ua0897,Restricted to declared computer science majors...,1 - 4
96,csci-ua0898,Restricted to declared computer science majors...,1 - 4
97,csci-ua0997,Permission of the department. Does not satisfy...,1 - 4
98,csci-ua0998,Permission of the department. Does not satisfy...,1 - 4


In [134]:
catalog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Number   100 non-null    object
 1   PreReqs  100 non-null    object
 2   Points   99 non-null     object
dtypes: object(3)
memory usage: 2.5+ KB


In [135]:
catalog_df.head(5)

,Number,PreReqs,Points
0,csci-ga1133,None,4
1,csci-ga1144,CSCI-GA 1133 or departmental permission.,4
2,csci-ga1170,At least one year of experience with a high-le...,3
3,csci-ga1180,None,3
4,csci-ga2110,Students taking this class should already have...,3


In [136]:
catalog_df.tail(5)

,Number,PreReqs,Points
95,csci-ua0897,Restricted to declared computer science majors...,1 - 4
96,csci-ua0898,Restricted to declared computer science majors...,1 - 4
97,csci-ua0997,Permission of the department. Does not satisfy...,1 - 4
98,csci-ua0998,Permission of the department. Does not satisfy...,1 - 4
99,frsem-ua0597,"Some programming experience in Python, Java, J...",4


In [137]:
catalog_df.sample(5)

,Number,PreReqs,Points
33,csci-ga2591,Students should have solid programming skills ...,3
78,csci-ua0421,"Computer Systems Organization (CSCI-UA 201), e...",4
69,csci-ua0061,Introduction to Computer Programming (CSCI-UA....,4
94,csci-ua0521,Permission of the department.,4
29,csci-ga2572,DS-GA 1001 Intro to Data Science or a graduate...,3


In [139]:
merged_df = pd.merge(schedule_df, catalog_df, on='Number', how='left')

In [142]:
merged_df = merged_df[['Number', 'Name', 'Instructor', 'Time', 'PreReqs', 'Points']]

In [143]:
merged_df

,Number,Name,Instructor,Time,PreReqs,Points
0,csci-ga1144,PAC II*,Mohamed Zahran,T 6:00-8:30PM,CSCI-GA 1133 or departmental permission.,4
1,csci-ga1144,PAC II Recitation,Gurkirat Singh Bajwa,R 7:10-8:00PM,CSCI-GA 1133 or departmental permission.,4
2,csci-ga1170,Fundamental Algorithms,Yevgeniy Dodis,T 7:10-9:00PM,At least one year of experience with a high-le...,3
3,csci-ga1170,Fundamental Algorithms Recitation,Alex Bienstock,R 8:10-9:00PM,At least one year of experience with a high-le...,3
4,csci-ga1170,Fundamental Algorithms Recitation,Harish Karthikeyan,R 8:10-9:00PM,At least one year of experience with a high-le...,3
...,...,...,...,...,...,...
147,csci-ua0480,Special Topics: Natural Language Processing,Adam Meyers,TR 9:30-10:45AM,Topics determine prerequisites.,4
148,csci-ua0480,Special Topics: Agile Software Development and...,Amos Bloomberg,MW 3:30-4:45PM,Topics determine prerequisites.,4
149,csci-ua0480,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 11:00-12:15PM,Topics determine prerequisites.,4
150,csci-ua0480,Special Topics: Algorithmic Problem Solving,Kunal Khatri,W 3:30-4:45PM,Topics determine prerequisites.,4


In [144]:
pd.set_option('display.max_rows', 200)

In [145]:
merged_df

,Number,Name,Instructor,Time,PreReqs,Points
0,csci-ga1144,PAC II*,Mohamed Zahran,T 6:00-8:30PM,CSCI-GA 1133 or departmental permission.,4
1,csci-ga1144,PAC II Recitation,Gurkirat Singh Bajwa,R 7:10-8:00PM,CSCI-GA 1133 or departmental permission.,4
2,csci-ga1170,Fundamental Algorithms,Yevgeniy Dodis,T 7:10-9:00PM,At least one year of experience with a high-le...,3
3,csci-ga1170,Fundamental Algorithms Recitation,Alex Bienstock,R 8:10-9:00PM,At least one year of experience with a high-le...,3
4,csci-ga1170,Fundamental Algorithms Recitation,Harish Karthikeyan,R 8:10-9:00PM,At least one year of experience with a high-le...,3
5,csci-ga1180,Mathematical Techniques For CS Applications,Parijat Dube,F 7:10-9:00PM,None,3
6,csci-ga2110,Programming Languages,Cory Plock,M 7:10-9:00PM,Students taking this class should already have...,3
7,csci-ga2110,Programming Languages Recitation,Monika Dagar,R 7:10-8:00PM,Students taking this class should already have...,3
8,csci-ga2110,Programming Languages Recitation,Tanmay Gatle,R 7:10-8:00PM,Students taking this class should already have...,3
9,csci-ga2110,Programming Languages,Yuxin Deng,T 1:15-3:05PM,Students taking this class should already have...,3


## Conclusion

1. did you spot any anomalies, discrepancies, or unexpected data or relationships between data? if so, in a markdown cell, describe any problem(s) you saw


Some of the anomalies or discrepancies I noticed were the instructor. The data frame above did not fully show the instructor names. For example, there were some courses that had more than one instructor. However, the programming I did only included the first name that appeared. Furthermore, another anomalies I can discover is from the 'Points' column. Although it makes sense to have 'None' as the value, it does not make sense to have 'NaN' as a value. If you take a look at some of the values from 'Points' column, you can observe a few 'NaN'.


2. additionally, describe how you might fix them (or if you already fixed them!)


As for the 'Points' columnm, I would fill in na using the .fillna method. It would be more appropriate to have "None" as the value. So I would fill in those NaN values with "None" and set the inplace as true.


3. lastly, based on the resulting DataFrame, describe the behavior of how=left on these particular DataFrames


Looking at the resulting DataFrame, the new merged dataframe contained all the information from the 'schedule dataframe.' For example, Adv Lab MS * were courses that could be seen in NYU schedule page only. Since we made the behavior to how=left, the newly merged dataframe contained information about Adv lab MS. The only major difference between schedule datafram and catalog datframe is the 'point' column. So even if there are course names that did not have points, they will still be available in the merged data frame.